In [1]:
import zipfile
import numpy as np
import pandas
from pprint import pprint

In [2]:
zip_ref = zipfile.ZipFile('1927eve.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

zip_ref = zipfile.ZipFile('1938eve.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()


In [36]:
filename = '1927DET.EVA'
gamelogs = []
gamecount = -1

for line in open(filename).readlines():
    linearray = line.split(',')
#    print(linearray)
    if (linearray[0] == 'id'):
        # 'id' starts the next game, so immediately advance gamecount  
        gamecount = gamecount + 1
        
        # record prior game data by adding it to the gamelogs list, unless gamecount = -1
        if (gamecount > 0):
            thisgamedict['info'] = info_dict
            gamelogs.append(thisgamedict)
        thisgamedict = {}
        info_dict = {}
        start_dict = {}
        vis_start_list = []
        home_start_list = []
        play_dict = {}
        data_dict = {}
        
        thisgamedict['id'] = linearray[1] 
        if (gamecount > 0):
            print(thisgamedict['id'],' had {:.0f} batters.'.format(batterscount))
        batterscount = 0
    if (linearray[0] == 'version'):
        thisgamedict['version'] = linearray[1]
    if (linearray[0] == 'info'):
        info_dict[linearray[1]] = linearray[2]
    if (linearray[0] == 'start'):
        batterscount = batterscount + 1
        
#        for i in range(1,len(linearray)):
#            batterscount = batterscount + 1
#            print(i,': ',linearray[i])
#pprint(gamelogs)

#pprint(infodict)
#pprint(infos)    
#pprint(infos)

DET192704290
  had 18 batters.
DET192704300
  had 18 batters.
DET192705010
  had 18 batters.
DET192705020
  had 18 batters.
DET192705030
  had 18 batters.
DET192705040
  had 18 batters.
DET192705050
  had 18 batters.
DET192705100
  had 18 batters.
DET192705110
  had 18 batters.
DET192705120
  had 18 batters.
DET192705130
  had 18 batters.
DET192705160
  had 18 batters.
DET192705170
  had 18 batters.
DET192705260
  had 18 batters.
DET192705270
  had 18 batters.
DET192705280
  had 18 batters.
DET192705301
  had 18 batters.
DET192705302
  had 18 batters.
DET192705310
  had 18 batters.
DET192706010
  had 18 batters.
DET192706270
  had 18 batters.
DET192706281
  had 18 batters.
DET192706282
  had 18 batters.
DET192706290
  had 18 batters.
DET192707081
  had 18 batters.
DET192707082
  had 18 batters.
DET192707091
  had 18 batters.
DET192707092
  had 18 batters.
DET192707100
  had 18 batters.
DET192707110
  had 18 batters.
DET192707160
  had 18 batters.
DET192707170
  had 18 batters.
DET19270